## Remember to save this as a new notebook before you begin solving!!
## Also remember to open the notebook through a virtual env that works well with keras

### This exercise is meant to teach you how to a powerful, existing neural network for a new problem in a similar domain. This exercise should come after you have some experience with NN (not neccessarily extensive experience)

The core of the exercise is downloading a trained NN for face classification, and using transfer learning techniques to solve a new problem. The new problem you will solve is classification of faces - automatic labeling of the different members of your family.

### Authors: Philip Tannor, Gal Eyal

# Creating your data 

In this exercise, you will get your hands dirty and create your own data:
1. Start by creating a folder with many photos of yourself (between 100 and 300).
2. Create similar folders for a number of roommates or family members (preferably people who live with you). Try to make sure the folders have similar amounts of photos in them.
3. Use the python program cropall.py (*works on linux only!!*), which should be in the folder of this exercise, to crop out the faces from the different photos. You should create a x/y ratio of 3/4, don't demand a perfect pixel ratio, and change the x/y ratio to 4/3 when you have a rotated photo. I you're working correctly, you should only be using the mouse click, the mouse scroll, and Spacebar.
4. Make sure to save the cropped photos of each person in a different folder. Notice that by default the crops will all go to the same folder, so find a way to deal with this manually.

# Imports and parameters

We left this section almost exactly as it was were when we created the solution, just to save some time. Don't feel bonded by these libraries (or parameters).

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import numpy as np
#These next 2 imports depend on the 'keras-vggface' folder in https://github.com/rcmalli/keras-vggface
#Just in case the git won't be around, we put this folder in the directory you're working in.
#Notice this if you move the notebook around.
# from keras_vggface import VGGFace
# from keras_vggface import utils
from keras.preprocessing import image
import keras
import unittest
import pandas as pd
from os import listdir,path
from sklearn.model_selection import train_test_split
#This next import will help you with augmentation - generating augmented photos from your originals.
#Read about this general teqnique, and also about 
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense
from keras.models import Model

In [5]:
input_size=(224, 224)
n_channels = 3

In [6]:
keras.backend.set_image_dim_ordering('tf')

AttributeError: module 'keras.backend' has no attribute 'set_image_dim_ordering'

## Try testing one of the existing models.
1. Create an instance of the VGGFace class you imported, and chose which model it will be using.
2. Use keras.preprocessing.image to read the only picture in the 'image' file (fix the target size).
3. Use vgg-keras.utils.preprocess_input for preprocessing the photo. You may have to expand the dimensions before this.
4. Use the model to create a prediction for the image.
5. Check how the model did. The guy you looked at should be indexed first.
6. Restart the kernel and next time skip this section (so that you aren't wasting RAM).

In [7]:
# vggface = VGGFace(model ='vgg16')

vggface = keras.applications.VGG16()

553467904/553467096 [==============================] - 34s 0us/step


In [8]:
vggface.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [32]:
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import decode_predictions

image = load_img('image/ajb.jpg', target_size=input_size)
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)

print(image.shape)
yhat = vggface.predict(image)

label = decode_predictions(yhat)
label = label[0][0]

print('%s (%.2f%%)' % (label[1], label[2]*100))

(1, 224, 224, 3)
suit (12.38%)


## Reading and preprocessing the data you created
create a DataFrame with 2 columns:
1. The exact path of each image
2. The label of each image

Split this data into train and test sets

In [11]:
df = pd.DataFrame(columns=['image_path', 'label'])
for folder in listdir('./data'):
    for im in listdir(path.join('./data', folder)):
        df = df.append({'image_path': path.join('./data', folder, im),
                  'label': folder}, ignore_index=True)

In [12]:
df.shape

(265, 2)

In [13]:
df.head()

,image_path,label
0,./data\gal\20150221_112656_HDR.jpg,gal
1,./data\gal\20150221_124226_HDR.jpg,gal
2,./data\gal\20150221_133047_HDR.jpg,gal
3,./data\gal\20150221_135339_HDR.jpg,gal
4,./data\gal\20150227_162125_HDR.jpg,gal


In [14]:
data_train, data_test = train_test_split(df, test_size=0.2)

Now use iterate through the dataframes and read the images corresponding to each row (using keras.preprocessing.image). Save the images as numpy arrays, and once again use utils.preprocess_input to preprocess the arrays.
Save the labels in 2 different ways:
1. Onehot encoding - for using a NN
2. Serial number of the corresponding class (one number per label) - for using an SVM

In [28]:
train_x = np.zeros((len(data_train), input_size[0], input_size[1], n_channels))

for i, row in enumerate(data_train.itertuples()):
    image_path = row.image_path
    
    image = load_img(image_path, target_size=input_size)
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    # Shape: 1 x 224 x 224 x 3
    
    train_x[i, :, :, :] = image

train_y_NN = pd.get_dummies(data_train['label'])

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_y_SVM = encoder.fit_transform(data_train['label'])

In [30]:
print('train_x', train_x.shape)
print('train_y_NN', train_y_NN.shape)
print('train_y_SVM', train_y_SVM.shape)

train_x (212, 224, 224, 3)
train_y_NN (212, 2)
train_y_SVM (212,)


In [33]:
test_x = np.zeros((len(data_test), input_size[0], input_size[1], n_channels))

for i, row in enumerate(data_test.itertuples()):
    image_path = row.image_path
    
    image = load_img(image_path, target_size=input_size)
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    # Shape: 1 x 224 x 224 x 3
    
    test_x[i, :, :, :] = image

test_y_NN = pd.get_dummies(data_test['label'])

encoder = LabelEncoder()
test_y_SVM = encoder.fit_transform(data_test['label'])

In [34]:
print('test_x', test_x.shape)
print('test_y_NN', test_y_NN.shape)
print('test_y_SVM', test_y_SVM.shape)

test_x (53, 224, 224, 3)
test_y_NN (53, 2)
test_y_SVM (53,)


Create an instance of keras.preprocessing.image.ImageDataGenerator, which will define how you will create augmentations of each original image you've created. Chose all the parameters on your own.

In [31]:
train_gen = ImageDataGenerator(rotation_range=90, width_shift_range=0.1, 
                               height_shift_range=0.1,zoom_range=0.2, 
                               horizontal_flip=True)

# Finally - transfer learning. 
### This should be done with the keras functional API (not Sequential)
1. Create an instance of VGGFace with the model of your choice. 
2. Examine the model's architecture using .summary(). Understand what it means to replace the last 2 dense layers (including the final softmax layer). 
3. Use .get_layer() to retrieve the last layer which you want to keep in your new NN.
4. Create the 2 new Dense layers which continue the previous pretrained layers (the last layer should have a softmax activation).
5. Create a new model which the input of the original model as input, and outputs the new dense-softmax layer. You can use .input on the old model for this.
6. Freeze all of the layers except the last 2 using .layers on the new model, and .trainable = False. This will stop you from training those layers.
7. Compile the model ('adam' worked ok for us as an optimizer).

In [46]:
nb_class = 2
hidden_dim = 512

vgg_model = keras.applications.VGG16()
dense_1 = vgg_model.get_layer('fc2').output

new_dense_1 = Dense(hidden_dim, activation='relu', name='fc3')(dense_1)
out = Dense(nb_class, activation='softmax', name='fc4')(new_dense_1)
custom_vgg_model = Model(vgg_model.input, out)

In [47]:
for layer in custom_vgg_model.layers[:-2]:
    print(layer.name)
    layer.trainable = False

input_4
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
flatten
fc1
fc2


In [48]:
custom_vgg_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [49]:
custom_vgg_model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

Now you're ready to train the model:
1. Use .fit_generator() and not .fit(), since you'll be using the augmentor you created.
2. Use .flow() on the instance of ImageDataGenerator as the first input. 
3. Choose a combination on batch_size(within .flow) and steps_per_epoch which will create a total number of images that you want per each epoch. 
4. Use validation data which isn't augmented.

How well is it doing? We got an accuracy of 90%-95% percent (validation data) on photos on the 2 of us (just 2 classes).

In [50]:
custom_vgg_model.fit_generator(train_gen.flow(train_x, train_y_NN, batch_size=20, seed=77), 
                               steps_per_epoch=13, epochs=100, validation_data=(test_x, test_y_NN), 
                               workers=50)

Epoch 1/100
13/13 [==============================] - 38s 3s/step - loss: 1.4762 - accuracy: 0.7951 - val_loss: 0.4824 - val_accuracy: 0.9057
Epoch 2/100
13/13 [==============================] - 34s 3s/step - loss: 0.6039 - accuracy: 0.8571 - val_loss: 0.0827 - val_accuracy: 0.9623
Epoch 3/100
13/13 [==============================] - 37s 3s/step - loss: 0.2925 - accuracy: 0.8889 - val_loss: 0.0400 - val_accuracy: 1.0000
Epoch 4/100
13/13 [==============================] - 35s 3s/step - loss: 0.1986 - accuracy: 0.9246 - val_loss: 0.0892 - val_accuracy: 0.9623
Epoch 5/100
13/13 [==============================] - 34s 3s/step - loss: 0.1483 - accuracy: 0.9524 - val_loss: 0.0190 - val_accuracy: 1.0000
Epoch 6/100
13/13 [==============================] - 33s 3s/step - loss: 0.1126 - accuracy: 0.9563 - val_loss: 0.0356 - val_accuracy: 0.9811
Epoch 7/100
13/13 [==============================] - 33s 3s/step - loss: 0.1271 - accuracy: 0.9444 - val_loss: 0.0637 - val_accuracy: 0.9623
Epoch 8/100
1

Epoch 59/100
13/13 [==============================] - 33s 3s/step - loss: 0.0153 - accuracy: 0.9960 - val_loss: 0.2139 - val_accuracy: 0.9623
Epoch 60/100
13/13 [==============================] - 35s 3s/step - loss: 0.0192 - accuracy: 0.9960 - val_loss: 0.3402 - val_accuracy: 0.9623
Epoch 61/100
13/13 [==============================] - 33s 3s/step - loss: 0.0383 - accuracy: 0.9881 - val_loss: 0.1080 - val_accuracy: 0.9811
Epoch 62/100
13/13 [==============================] - 33s 3s/step - loss: 0.0210 - accuracy: 0.9881 - val_loss: 0.2844 - val_accuracy: 0.9623
Epoch 63/100
13/13 [==============================] - 34s 3s/step - loss: 0.0288 - accuracy: 0.9921 - val_loss: 0.3090 - val_accuracy: 0.9623
Epoch 64/100
13/13 [==============================] - 41s 3s/step - loss: 0.0396 - accuracy: 0.9841 - val_loss: 0.3822 - val_accuracy: 0.9623
Epoch 65/100
13/13 [==============================] - 34s 3s/step - loss: 0.0596 - accuracy: 0.9762 - val_loss: 0.0445 - val_accuracy: 0.9811
Epoch 

# Similar exercise with SVM
1. Now create a new model which outputs the data from the last frozen layer. Use this model to create feature for each image.
2. Create a SVM classifier using these features. We used sklearn.svm.SVC.
3. Did this work better or less good? Evaluate using sklearn.metrics.classification_report or sklearn.metrics.accuracy_score.
4. Try the same thing with the one-before-last layer.

# Congrats! You completed the exercise. 
# Bonus - use one of these models and a webcam to created a personalized security system for your house.